## Chat labeling pipeline


In [56]:
import pandas as pd
import re
import json
import csv

### Ingest raw chat data

In [57]:
# chats = pd.read_csv('data_inputs/test_export.csv')
# chats_df = pd.read_json('data_inputs/export1.json', lines=True)

chats = []
for line in open('data_inputs/chat_export1.json', 'r'):
    chats.append(json.loads(line))

In [58]:
chats[1:3]

[{'_id': {'$oid': '5c7acb202f46c1066e31fd5b'},
  'student_id': 'UFR6C3N5B',
  'time': 'Sat Mar 02 2019 18:27:10 GMT+0000 (Coordinated Universal Time)',
  'topic': 'live session',
  'course': 'a',
  'prof_name': 'a',
  'instr_wk': 'a',
  'emoji_sentiment': 'pos1',
  'topic_response': 'a'},
 {'_id': {'$oid': '5c7ae81e2f46c1066e31fd5c'},
  'student_id': 'UFR6C3N5B',
  'time': 'Sat Mar 02 2019 18:27:10 GMT+0000 (Coordinated Universal Time)',
  'topic': 'office hours',
  'course': '203',
  'prof_name': 'john do',
  'instr_wk': '2',
  'emoji_sentiment': 'neg',
  'topic_response': 'not much'}]

### Clean data


In [59]:
def sentiment_transform(x):
    return {
        'neg1':1,
        'neg' :2,
        'neu' :3,
        'pos' :4,
        'pos1':5
    }[x]



In [60]:
chats[1]

{'_id': {'$oid': '5c7acb202f46c1066e31fd5b'},
 'student_id': 'UFR6C3N5B',
 'time': 'Sat Mar 02 2019 18:27:10 GMT+0000 (Coordinated Universal Time)',
 'topic': 'live session',
 'course': 'a',
 'prof_name': 'a',
 'instr_wk': 'a',
 'emoji_sentiment': 'pos1',
 'topic_response': 'a'}

In [61]:
for chat in chats:
    chat['emoji_sentiment_int']= sentiment_transform(chat['emoji_sentiment'])

In [62]:
chats[1]

{'_id': {'$oid': '5c7acb202f46c1066e31fd5b'},
 'student_id': 'UFR6C3N5B',
 'time': 'Sat Mar 02 2019 18:27:10 GMT+0000 (Coordinated Universal Time)',
 'topic': 'live session',
 'course': 'a',
 'prof_name': 'a',
 'instr_wk': 'a',
 'emoji_sentiment': 'pos1',
 'topic_response': 'a',
 'emoji_sentiment_int': 5}

### Apply topic models to chat data


In [63]:
def pred_topic(text):
    return text

def pred_sub_topic(text):
    return text


In [64]:
for chat in chats:
    chat['pred_topic'] = pred_topic(chat['topic_response'])
    chat['pred_sub_topic'] = pred_sub_topic(chat['topic_response'])

### Export appended data

In [67]:
with open('data_outputs/updated_chatdata.csv', 'w') as f:  # Just use 'w' mode in 3.x
    w = csv.DictWriter(f, chats[1].keys())
    w.writeheader()
    for chat in chats:
        w = csv.DictWriter(f, chat.keys())
        w.writerow(chat)